# 1. Import Libraries

In [1]:
import pandas as pd
import pandas.api.types as ptypes
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
import math, os, shutil
warnings.filterwarnings('ignore')
mpl.rcParams['figure.dpi'] = 500
pd.set_option('display.max_columns', 1000)
%matplotlib inline

# Problem Statement

Congratulations – you have been hired as Chief Data Scientist of MedCamp – a not for profit organization dedicated in making health conditions for working professionals better. MedCamp was started because the founders saw their family suffer due to bad work life balance and neglected health.

MedCamp organizes health camps in several cities with low work life balance. They reach out to working people and ask them to register for these health camps. For those who attend, MedCamp provides them facility to undergo health checks or increase awareness by visiting various stalls (depending on the format of camp).

MedCamp has conducted 65 such events over a period of 4 years and they see a high drop off between “Registration” and Number of people taking tests at the Camps. In last 4 years, they have stored data of ~110,000 registrations they have done.

One of the huge costs in arranging these camps is the amount of inventory you need to carry. If you carry more than required inventory, you incur unnecessarily high costs. On the other hand, if you carry less than required inventory for conducting these medical checks, people end up having bad experience.

## Process

- MedCamp employees / volunteers reach out to people and drive registrations.
- During the camp, People who “ShowUp” either undergo the medical tests or visit stalls depending on the format of healthcamp.

## Note 

- Since this is a completely voluntary activity for the working professionals, MedCamp usually has little profile information about these people.
- For a few camps, there was hardware failure, so some information about date and time of registration is lost.
- MedCamp runs 3 formats of these camps. The first and second format provides people with an instantaneous health score. The third format provides information about several health issues through various awareness stalls.

## Favorable outcome:
- For the first 2 formats, a favourable outcome is defined as getting a health_score, while in the third format it is defined as visiting at least a stall.
- You need to predict the chances (probability) of having a favourable outcome.

# Data Description

## Train Data
train.zip contains 6 different csv files apart from the data dictionary as described below:

**Health_Camp_Detail.csv** – File containing Health_Camp_Id, Camp_Start_Date, Camp_End_Date and Category details of each camp.

**Train.csv** – File containing registration details for all the test camps. This includes Patient_ID, Health_Camp_ID, Registration_Date and a few anonymized variables as on registration date.

**Patient_Profile.csv** – This file contains Patient profile details like Patient_ID, Online_Follower, Social media details, Income, Education, Age, First_Interaction_Date, City_Type and Employer_Category

**First_Health_Camp_Attended.csv** – This file contains details about people who attended health camp of first format. This includes Donation (amount) & Health_Score of the person.

**Second_Health_Camp_Attended.csv** - This file contains details about people who attended health camp of second format. This includes Health_Score of the person.

**Third_Health_Camp_Attended.csv** - This file contains details about people who attended health camp of third format. This includes Number_of_stall_visited & Last_Stall_Visited_Number.

## Test Data

**Test.csv** – File containing registration details for all the camps done after 1st April 2006. This includes Patient_ID, Health_Camp_ID, Registration_Date and a few anonymized variables as on registration date. Participant should make predictions for these patient camp combinations

## Submission File

**sample_submission.csv**

**Patient_ID**: Unique Identifier for each patient. This ID is not sequential in nature and can not be used in modeling

**Health_Camp_ID**: Unique Identifier for each camp. This ID is not sequential in nature and can not be used in modeling

**Outcome**: Predicted probability for having a favourable outcome depending on the format

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import FunctionTransformer

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [4]:
from sklearn.metrics import f1_score, auc, plot_confusion_matrix, plot_roc_curve

# Load Data

In [5]:
train_data = pd.read_csv('../Data/Train/Train.csv')
firstcamp_data = pd.read_csv('../Data/Train/First_Health_Camp_Attended.csv')
secondcamp_data = pd.read_csv('../Data/Train/Second_Health_Camp_Attended.csv')
thirdcamp_data = pd.read_csv('../Data/Train/Third_Health_Camp_Attended.csv')
healthcamp_data = pd.read_csv('../Data/Train/Health_Camp_Detail.csv')
patient_data = pd.read_csv('../Data/Train/Patient_Profile.csv')

test_data = pd.read_csv('../Data/test_l0Auv8Q.csv')
submission_data = pd.read_csv('../Data/sample_submmission.csv')

# Data Cleaning

In [6]:
train_final_data = pd.merge(train_data, firstcamp_data.drop('Unnamed: 4', axis=1), how='left', on=['Patient_ID', 'Health_Camp_ID'], indicator='camp1_merge_ind')
train_final_data = pd.merge(train_final_data, secondcamp_data, how='left', on=['Patient_ID', 'Health_Camp_ID'], indicator='camp2_merge_ind')
train_final_data = pd.merge(train_final_data, thirdcamp_data, how='left', on=['Patient_ID', 'Health_Camp_ID'], indicator='camp3_merge_ind')
train_final_data = pd.merge(train_final_data, healthcamp_data, how='left', on='Health_Camp_ID', indicator='healthcamp_merge_ind')
train_final_data = pd.merge(train_final_data, patient_data, how='left', on='Patient_ID', indicator='patient_merge_ind')

In [7]:
train_final_data['Outcome'] = 0
train_final_data.loc[(train_final_data['camp1_merge_ind']=='both') | 
                     (train_final_data['camp2_merge_ind']=='both') |
                     ((train_final_data['camp3_merge_ind']=='both') & (train_final_data['Number_of_stall_visited']>0))
                     ,'Outcome'] = 1

In [8]:
test_final_data = pd.merge(test_data, firstcamp_data.drop('Unnamed: 4', axis=1), how='left', on=['Patient_ID', 'Health_Camp_ID'], indicator='camp1_merge_ind')
test_final_data = pd.merge(test_final_data, secondcamp_data, how='left', on=['Patient_ID', 'Health_Camp_ID'], indicator='camp2_merge_ind')
test_final_data = pd.merge(test_final_data, thirdcamp_data, how='left', on=['Patient_ID', 'Health_Camp_ID'], indicator='camp3_merge_ind')
test_final_data = pd.merge(test_final_data, healthcamp_data, how='left', on='Health_Camp_ID', indicator='healthcamp_merge_ind')
test_final_data = pd.merge(test_final_data, patient_data, how='left', on='Patient_ID', indicator='patient_merge_ind')

In [9]:
date_cols = ['Registration_Date', 'Camp_Start_Date', 'Camp_End_Date', 'First_Interaction']

In [10]:
def to_date(df):
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format='%d-%b-%y')
    return df

In [11]:
train_final_data = to_date(train_final_data)
test_final_data = to_date(test_final_data)

In [12]:
num_cols = ['Income', 'Education_Score', 'Age']

In [13]:
def to_numeric(df, columns):
    for col in df.columns:
        if (col in num_cols) & ~(ptypes.is_numeric_dtype(df[col])):
            df[col] = df[col].replace({'None':''})
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

In [14]:
train_final_data = to_numeric(train_final_data, num_cols)
test_final_data = to_numeric(test_final_data, num_cols)

## Drop unnecessary columns

In [16]:
unnec_cols = ['camp1_merge_ind', 'camp2_merge_ind', 'camp3_merge_ind', 'healthcamp_merge_ind', 'patient_merge_ind']

In [17]:
train_final_data = train_final_data.drop(unnec_cols, axis=1)
test_final_data = test_final_data.drop(unnec_cols, axis=1)

## Missing Imputations

In [23]:
from sklearn.impute import SimpleImputer

### Mean Imputation

In [42]:
mean_impute_cols = ['Age']
imp_mean = SimpleImputer(strategy='mean')
imp_mean.fit(train_final_data[mean_impute_cols])
train_final_data[mean_impute_cols] = imp_mean.transform(train_final_data[mean_impute_cols])
test_final_data[mean_impute_cols] = imp_mean.transform(test_final_data[mean_impute_cols])

### Frequent Imputation

In [43]:
freq_impute_cols = ['Income', 'Education_Score', 'City_Type', 'Employer_Category']
imp_freq = SimpleImputer(strategy='most_frequent')
imp_freq.fit(train_final_data[freq_impute_cols])
train_final_data[freq_impute_cols] = imp_freq.transform(train_final_data[freq_impute_cols])
test_final_data[freq_impute_cols] = imp_freq.transform(test_final_data[freq_impute_cols])

### Zero Imputation

In [44]:
zero_impute_cols = ['Donation', 'Health_Score', 'Health Score', 'Number_of_stall_visited', 'Last_Stall_Visited_Number']
train_final_data[zero_impute_cols] = train_final_data[zero_impute_cols].fillna(0)
test_final_data[zero_impute_cols] = test_final_data[zero_impute_cols].fillna(0)

## Constants

In [19]:
ID1 = 'Patient_ID'
ID2 = 'Health_Camp_ID'
target = 'Outcome'
date_columns = ['Registration_Date', 'Camp_Start_Date', 'Camp_End_Date', 'First_Interaction']
discrete_columns = ['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Category1', 'Category2', 'Category3', 'Online_Follower', 
                   'LinkedIn_Shared', 'Twitter_Shared', 'Facebook_Shared', 'City_Type', 'Employer_Category']

In [48]:
train_final_data.columns

Index(['Patient_ID', 'Health_Camp_ID', 'Registration_Date', 'Var1', 'Var2',
       'Var3', 'Var4', 'Var5', 'Donation', 'Health_Score', 'Health Score',
       'Number_of_stall_visited', 'Last_Stall_Visited_Number',
       'Camp_Start_Date', 'Camp_End_Date', 'Category1', 'Category2',
       'Category3', 'Online_Follower', 'LinkedIn_Shared', 'Twitter_Shared',
       'Facebook_Shared', 'Income', 'Education_Score', 'Age',
       'First_Interaction', 'City_Type', 'Employer_Category', 'Outcome'],
      dtype='object')

In [46]:
random_state = 1234

In [47]:
should_ohe = True
should_scale = True

## Scaling

In [ ]:
if should_scale:
    for col in train_final_data.columns:
        